In [2]:
# Question: Do detached() tensors track their own gradients seperately?
# Ans: Yes!
# https://discuss.pytorch.org/t/why-is-the-clone-operation-part-of-the-computation-graph-is-it-even-differentiable/67054/11

import torch

a = torch.tensor([2.0], requires_grad=True)
b = a.detach()
b.requires_grad = True

la = (5.0 - a)**2
la.backward()
print(f'a.grad = {a.grad}')

lb = (6.0 - b)**2
lb.backward()
print(f'b.grad = {b.grad}')

a.grad = tensor([-6.])
b.grad = tensor([-8.])


In [1]:
import torch
import torch.nn as nn

from collections import OrderedDict

params = OrderedDict([
    ('fc0', nn.Linear(in_features=4,out_features=4)),
    ('ReLU0', nn.ReLU()),
    ('fc1', nn.Linear(in_features=4,out_features=1))
])
mdl = nn.Sequential(params)

print(params)
print(mdl._parameters)
print(params == params)
print(mdl._parameters == params)
print(mdl._modules)

print()
for name, w in mdl.named_parameters():
    print(name, w.norm(2))

print()    
#mdl._modules['fc0'] = nn.Linear(10,11)
mdl._modules[0]

for name, w in mdl.named_parameters():
    print(name, w.norm(2))

OrderedDict([('fc0', Linear(in_features=4, out_features=4, bias=True)), ('ReLU0', ReLU()), ('fc1', Linear(in_features=4, out_features=1, bias=True))])
OrderedDict()
True
False
OrderedDict([('fc0', Linear(in_features=4, out_features=4, bias=True)), ('ReLU0', ReLU()), ('fc1', Linear(in_features=4, out_features=1, bias=True))])

fc0.weight tensor(1.2500, grad_fn=<NormBackward0>)
fc0.bias tensor(0.6285, grad_fn=<NormBackward0>)
fc1.weight tensor(0.6283, grad_fn=<NormBackward0>)
fc1.bias tensor(0.3797, grad_fn=<NormBackward0>)



KeyError: 0

In [19]:
## Q: are parameters are in computation graph?
import torch
import torch.nn as nn
from torchviz import make_dot

from collections import OrderedDict

fc0 = nn.Linear(in_features=3,out_features=1)
params = [('fc0', fc0)]
mdl = nn.Sequential(OrderedDict(params))

x = torch.randn(1,3)
y = torch.randn(1)

l = ( mdl(x) - y )**2

# make_dot(l,{x:'x',y:'y','fc0':fc0})
print(fc0.weight)
print(fc0.bias)
print(fc0.weight.to_tens)
print()
# make_dot(l,{x:'x',y:'y','fc0':fc0})
make_dot(l,{'x':x,'y':y})
make_dot(l)

Parameter containing:
tensor([[-0.1336, -0.5492,  0.3745]], requires_grad=True)
Parameter containing:
tensor([0.1751], requires_grad=True)



KeyError: 4866905824

In [8]:
'''
expand
'''

import torch

x = torch.randn([2,3,4,5])

# h_0 of shape (num_layers * num_directions, batch, hidden_size)
h = torch.randn([1,4,8])

x_mean = x.mean()
print(x_mean.size())
print(x_mean)
x = x_mean.expand_as(h)
print(x.size())
print(x)

torch.Size([])
tensor(0.1607)
torch.Size([1, 4, 8])
tensor([[[0.1607, 0.1607, 0.1607, 0.1607, 0.1607, 0.1607, 0.1607, 0.1607],
         [0.1607, 0.1607, 0.1607, 0.1607, 0.1607, 0.1607, 0.1607, 0.1607],
         [0.1607, 0.1607, 0.1607, 0.1607, 0.1607, 0.1607, 0.1607, 0.1607],
         [0.1607, 0.1607, 0.1607, 0.1607, 0.1607, 0.1607, 0.1607, 0.1607]]])


In [15]:
import torch

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
type(device)
print(device == 'cpu')
device.type

cpu
False


'cpu'

In [10]:
# THIS WORKS

from torch.utils.tensorboard import SummaryWriter

from pathlib import Path


#log_dir (string) – Save directory location. 
#Default is runs/CURRENT_DATETIME_HOSTNAME, which changes after each run.

tb = SummaryWriter()
tb.add_scalar('loss', 111)

In [21]:
from torch.utils.tensorboard import SummaryWriter

from pathlib import Path


def CURRENT_DATETIME_HOSTNAME(comment=''):
    #if not log_dir:
    import socket
    import os
    from datetime import datetime
    current_time = datetime.now().strftime('%b%d_%H-%M-%S')
    log_dir = os.path.join('runs', current_time + '_' + socket.gethostname() + comment)
    return Path(log_dir)

#log_dir (string) – Save directory location. 
#Default is runs/CURRENT_DATETIME_HOSTNAME, which changes after each run.
# tensorboard --logdir=runs
log_dir = (Path('~/automl-meta-learning/') / CURRENT_DATETIME_HOSTNAME()).expanduser()
print(log_dir)
tb = SummaryWriter(log_dir=log_dir)
tb.add_scalar('loss', 15)

/Users/brandomiranda/automl-meta-learning/runs/Mar12_14-22-43_wirelessprv-10-193-0-218.near.illinois.edu


In [6]:
# download mini-imagenet automatically

#from torchvision.utils import download_and_extract_archive

import torchvision.utils as utils

print(utils)
#print(download_and_extract_archive)

<module 'torchvision.utils' from '/Users/brandomiranda/miniconda3/envs/automl-meta-learning/lib/python3.7/site-packages/torchvision/utils.py'>


AttributeError: module 'torchvision.utils' has no attribute 'download_and_extract_archive'

In [14]:
#torch concat, https://pytorch.org/docs/stable/torch.html#torch.cat
# Concatenates the given sequence of seq tensors in the given dimension. 
# All tensors must either have the same shape (except in the concatenating dimension) or be empty.
import torch

g1 = torch.randn(3,2)
g2 = torch.randn(4,2)

g3 = torch.randn(4,2,3)

grads = [g1, g2]
print(g1.view(-1).size())
print(g2.view(-1).size())
print(g3.view(-1).size())
#print(g3.view(-1))

grads = torch.cat(grads, dim=0)
print(grads)
print(grads.size())
print(grads.mean())
print(grads.std())

# torch stack, https://pytorch.org/docs/stable/torch.html#torch.stack
# Concatenates sequence of tensors along a new dimension. 
# All tensors need to be of the same size.
# torch.stack([g1,g2], dim=0)

torch.Size([6])
torch.Size([8])
torch.Size([24])
tensor([[-0.3866,  0.9033],
        [-1.2136,  0.0404],
        [-0.6670, -1.0157],
        [-0.5798, -0.0218],
        [-0.4215,  0.4758],
        [ 0.1414, -0.3985],
        [ 1.7760, -0.3438]])
torch.Size([7, 2])
tensor(-0.1223)
tensor(0.7795)


RuntimeError: invalid argument 0: Sizes of tensors must match except in dimension 0. Got 3 and 4 in dimension 1 at /Users/distiller/project/conda/conda-bld/pytorch_1579022061893/work/aten/src/TH/generic/THTensor.cpp:612

In [ ]:
import torch

a = torch.tensor([1,2,3.], requires_grad=True)
a_detached = a.detach()
print(a_detached.is_leaf)
a_detached_sum = a.sum()
print(c.is_leaf)
d = c.detach()
print(d.is_leaf)

In [6]:
import torch

from types import SimpleNamespace
from pathlib import Path
from pprint import pprint

x = torch.empty([1,2,3])
print(x.size())

args = SimpleNamespace()
args.data_root = "~/automl-meta-learning/data/miniImagenet"

#n1313361300001299.jpg
args.data_root = Path(args.data_root).expanduser()

torch.Size([1, 2, 3])


In [13]:
import torch

CHW = 3,12,12
x = torch.randn(CHW)
y = torch.randn(CHW)

new = [x,y]
new = torch.stack(new)
print(x.size())
print(new.size())

torch.Size([3, 12, 12])
torch.Size([2, 3, 12, 12])


In [14]:
print('a');print('b')

a
b
